### Importing Libraries

In [1]:
import pandas as pd
import numpy as np

from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from transformers import BertTokenizerFast as BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup, BertForMaskedLM, BertConfig

import pytorch_lightning as pl
from pytorch_lightning.metrics.functional import accuracy, f1, auroc
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

%matplotlib inline
%config InlineBackend.figure_format='retina'

RANDOM_SEED = 123

sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8

pl.seed_everything(RANDOM_SEED)

Global seed set to 123


123

In [ ]:
# Require to change: 
parent_dir = '~/Conv_BERT/task-specific-datasets/banking_data/'
train_path = parent_dir + 'train.csv'
test_path = parent_dir + 'test.csv'
logging_path = 'lightning_logs_test'
checkpoint_path = 'checkpoints_test'
BERT_MODEL_NAME = 'bert-base-cased'
MAX_TOKEN_COUNT = 300
N_EPOCHS = 5
BATCH_SIZE = 6
pretrained_checkpoint = "/h/elau/Conv_BERT/ConversationalAI/intentIdentification/predefinedClassification/checkpoints_banking/best-checkpoint.ckpt"

In [3]:
thisdict = {
    "balance not updated after cheque or cash deposit": "cash balance not updated",
    "wrong exchange rate for cash withdrawal": "wrong cash withdrawal", 
    "Refund not showing up": "refund problem",
    "balance not updated after bank transfer": "transfer balance not updated",
    "card about to expire": "expire card",
    "top up by cash or cheque": "top up cash",
    "direct debit payment not recognised": "payment not recognised",
    "disposable card limits": "card limits",
    "apple pay or google pay": "app pay", 
    "get disposable virtual card": "get virtual card",
    "card payment wrong exchange rate": "wrong exchange rate",
    "top up by card charge": "top up card", 
    "wrong amount of cash received": "wrong cash received",
    "beneficiary not allowed": "beneficiary", 
    "top up by bank transfer charge": "top up transfer", 
    "transfer not received by recipient": "transfer not received", 
    "supported cards and currencies": "card and currency", 
}

In [2]:
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
df_train = df_train.sample(frac=1)
df_test = df_test.sample(frac=1)

In [4]:
def print_model_info(model):
    # Get all of the model's parameters as a list of tuples.
    params = list(model.named_parameters())

    print('The BERT model has {:} different named parameters.\n'.format(len(params)))

    print('==== Embedding Layer ====\n')

    for p in params[0:5]:
        print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

    print('\n==== First Transformer ====\n')

    for p in params[5:21]:
        print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

    print('\n==== Output Layer ====\n')

    for p in params[-4:]:
        print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

In [5]:
df_train[100:110]

,text,category
6685,Help! I need to transfer something to China.,transfer_timing
9794,Can you tell me what I need for identity valid...,verify_my_identity
5340,I accidentally left my phone at the hotel so n...,lost_or_stolen_phone
1982,My top-up hasn't gone through,pending_top_up
9829,What documentation do you accept for the ident...,verify_my_identity
5450,How do I begin the process of a refund?,request_refund
5238,What is the pending payment on my card about?,pending_card_payment
797,My account says I have money but I withdrew it...,pending_cash_withdrawal
7850,I would like to purchase another card.,getting_spare_card
6410,A payment is showing twice. Why is that?,transaction_charged_twice


In [6]:
df_train['category'] = df_train['category'].str.replace('_', ' ')
df_train['category'] = df_train['category'].replace(thisdict, regex=True)

In [7]:
df_train[100:110]

,text,category
6685,Help! I need to transfer something to China.,transfer timing
9794,Can you tell me what I need for identity valid...,verify my identity
5340,I accidentally left my phone at the hotel so n...,lost or stolen phone
1982,My top-up hasn't gone through,pending top up
9829,What documentation do you accept for the ident...,verify my identity
5450,How do I begin the process of a refund?,request refund
5238,What is the pending payment on my card about?,pending card payment
797,My account says I have money but I withdrew it...,pending cash withdrawal
7850,I would like to purchase another card.,getting spare card
6410,A payment is showing twice. Why is that?,transaction charged twice


In [9]:
train_questions = list(df_train['text'])
train_intents = list(df_train['category'])
test_questions = list(df_test['text'])
test_intents = list(df_test['category'])

In [9]:
df_train.groupby('category').count()['text'].sort_values()

category
contactless not working      35
virtual card not working     41
card acceptance              59
card swallowed               61
lost or stolen card          82
                           ... 
cash withdrawal charge      177
wrong cash received         180
cash balance not updated    181
payment not recognised      182
card payment fee charged    187
Name: text, Length: 77, dtype: int64

In [10]:
unique_intents = []
for ele in train_intents: 
    if ele not in unique_intents:
        unique_intents.append(ele)
print(len(unique_intents))

77


In [14]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertForMaskedLM.from_pretrained('bert-base-cased')
print(len(tokenizer))
print(tokenizer.decode([28996]))

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


28996

In [16]:
special_tokens_dict = {'additional_special_tokens': ['[BLK]']}
tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))

Embedding(28997, 768)

In [17]:
print(len(tokenizer))
print(tokenizer.decode([28996]))

28997

In [19]:
# Check if the labels are more than 6 tokens
LABEL_COLUMNS = df_train['category'].unique()
for intent in LABEL_COLUMNS: 
    intent = intent.replace("_", " ")
    inputs = tokenizer(intent, return_tensors='pt')
    if len(inputs['input_ids'][0]) > 6: 
        print(intent)
        print(len(inputs['input_ids'][0]))

In [20]:
def preprocessing(questions, intents):
    text = []
    for i in range(len(questions)): 
        curr_intent = tokenizer(intents[i], return_tensors='pt')

        extra_token_curr = 6-(len(curr_intent["input_ids"][0])-2)
        text.append(questions[i] + " the reason of the question is " + intents[i] + " [BLK]"*extra_token_curr)
    print(text[0:2])
    return text

In [21]:
train_questions = preprocessing(train_questions, train_intents)
train_questions[0:5]

["I'm worried my card might be lost in the mail? How long does it usually take to arrive? the reason of the question is card delivery estimate [BLK] [BLK] [BLK]", "I got charged a fee that shouldn't be there from my cash the reason of the question is cash withdrawal charge [BLK] [BLK] [BLK]"]


["I'm worried my card might be lost in the mail? How long does it usually take to arrive? the reason of the question is card delivery estimate [BLK] [BLK] [BLK]",
 "I got charged a fee that shouldn't be there from my cash the reason of the question is cash withdrawal charge [BLK] [BLK] [BLK]",
 'Do you charge for making a withdrawal? I took some money out of my account earlier and I was charged for this. the reason of the question is cash withdrawal charge [BLK] [BLK] [BLK]',
 "Is there an issue with my account?  I don't see a cheque deposit that I made yesterday.  Please assist. the reason of the question is cash balance not updated [BLK] [BLK]",
 'Are there ways for other people to send me money? the reason of the question is receiving money [BLK] [BLK] [BLK] [BLK]',
 'How can I receive money? the reason of the question is receiving money [BLK] [BLK] [BLK] [BLK]',
 'where can I change my address? the reason of the question is edit personal details [BLK] [BLK] [BLK]',
 'What do I do i

In [22]:
text = train_questions[:100]

In [23]:
inputs = tokenizer(text, return_tensors='pt', max_length=300, truncation=True, padding='max_length')

{'input_ids': tensor([[ 101,  146,  112,  ...,    0,    0,    0],
        [ 101,  146, 1400,  ...,    0,    0,    0],
        [ 101, 2091, 1128,  ...,    0,    0,    0],
        ...,
        [ 101, 2091,  146,  ...,    0,    0,    0],
        [ 101,  159, 9866,  ...,    0,    0,    0],
        [ 101, 1327, 2344,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [24]:
inputs['labels'] = inputs.input_ids.detach().clone()
inputs['labels'][0][0:50]

tensor([  101,   146,   112,   182,  4472,  1139,  3621,  1547,  1129,  1575,
         1107,  1103,  6346,   136,  1731,  1263,  1674,  1122,  1932,  1321,
         1106,  6657,   136,  1103,  2255,  1104,  1103,  2304,  1110,  3621,
         6779, 10301, 28996, 28996, 28996,   102,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

In [35]:
class WOZDataset(Dataset):
    def __init__(
        self, 
        text, 
        tokenizer, 
        max_token_len: int = 300
    ):
        self.data = text
        self.tokenizer = tokenizer
        self.max_token_len = max_token_len

    def __getitem__(self, idx):
        data_row = self.data[idx]
#         print(data_row)
        inputs = self.tokenizer(data_row, return_tensors='pt', max_length=300, truncation=True, padding='max_length')
        inputs['labels'] = inputs.input_ids.detach().clone()
        for i in range(inputs.input_ids.shape[0]):
            curr_line = inputs.input_ids[i]
            for index in reversed(range(len(curr_line))): 
                if curr_line[index] != 0: 
                    curr_line[index-1] = 103
                    curr_line[index-2] = 103
                    curr_line[index-3] = 103
                    curr_line[index-4] = 103
                    curr_line[index-5] = 103
                    curr_line[index-6] = 103
                    break 
        return dict(input_ids = inputs.input_ids.flatten(), attention_mask = inputs.attention_mask.flatten(), labels=inputs.labels)
    def __len__(self):
        return len(self.data)

In [38]:
testing = WOZDataset(train_questions, tokenizer)
sample_item = testing[0]
sample_item.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [39]:
print(sample_item['labels'][0:50])
print(sample_item['input_ids'][0:50])
print(sample_item["input_ids"].shape)

tensor([[  101,   146,   112,   182,  4472,  1139,  3621,  1547,  1129,  1575,
          1107,  1103,  6346,   136,  1731,  1263,  1674,  1122,  1932,  1321,
          1106,  6657,   136,  1103,  2255,  1104,  1103,  2304,  1110,  3621,
          6779, 10301, 28996, 28996, 28996,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [42]:
sample_batch = next(iter(DataLoader(testing, batch_size=8, num_workers=2)))
sample_batch["input_ids"].shape, sample_batch["attention_mask"].shape

(torch.Size([8, 300]), torch.Size([8, 300]))

In [43]:
output = model(sample_batch["input_ids"], sample_batch["attention_mask"])

In [44]:
class WOZDataModule(pl.LightningDataModule):

  def __init__(self, train_df, test_df, tokenizer, batch_size=8, max_token_len=300):
    super().__init__()
    self.batch_size = batch_size
    self.train_df = train_df
    self.test_df = test_df
    self.tokenizer = tokenizer
    self.max_token_len = max_token_len

  def setup(self, stage=None):
    self.train_dataset = WOZDataset(
        self.train_df,
        self.tokenizer,
        self.max_token_len
    )

    self.test_dataset = WOZDataset(
      self.test_df,
      self.tokenizer,
      self.max_token_len
    )

  def train_dataloader(self):
    return DataLoader(
      self.train_dataset,
      batch_size=self.batch_size,
      shuffle=True, 
      num_workers=2
    )

  def val_dataloader(self):
    return DataLoader(
      self.test_dataset,
      batch_size=self.batch_size,
      num_workers=2
    )

  def test_dataloader(self):
    return DataLoader(
      self.test_dataset,
      batch_size=self.batch_size,
      num_workers=2
    )

In [45]:
N_EPOCHS = 40
BATCH_SIZE = 8

train_df = train_questions[:90]
val_df = train_questions[10:]

data_module = WOZDataModule(
  train_df,
  val_df, 
  tokenizer,
  batch_size=BATCH_SIZE,
  max_token_len=300
)

In [46]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)
model.resize_token_embeddings(len(tokenizer))

Embedding(28997, 768)

In [57]:
class LMModel(pl.LightningModule):
    def __init__(self, model_name_or_path, tokenizer):
        super().__init__()
        self.bert = BertForMaskedLM.from_pretrained(model_name_or_path, return_dict=True)
        self.bert.resize_token_embeddings(len(tokenizer))

    def forward(self, x):
        return self.bert(x).logits

    def training_step(self, batch, batch_idx):
        loss = self.bert(**batch).loss
        self.log('train_loss', loss, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.bert(**batch).loss
        self.log("val_loss", loss, on_epoch=True, logger=True)
        return loss
    
    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(),
                          lr=5e-5,
                          betas=(0.9,
                                 0.999),
                          eps=1e-8,)
        return optimizer

In [58]:
lmmodel = LMModel(
    model_name_or_path = 'bert-base-cased', 
    tokenizer = tokenizer
)
len(tokenizer)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The BERT model has 202 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (28997, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

28997

### Training

In [59]:
!rm -rf logging_path/
!rm -rf checkpoint_path/

%load_ext tensorboard
%tensorboard --logdir ./logging_path

In [60]:
checkpoint_callback = ModelCheckpoint(
  dirpath="checkpoints_OOD",
  filename="best-checkpoints",
  save_top_k=1,
  verbose=True,
  monitor="val_loss",
  mode="min"
)

In [61]:
logger = TensorBoardLogger(logging_path, name="masked_lm")

In [62]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=2)

In [63]:
trainer = pl.Trainer(
  logger=logger,
  checkpoint_callback=checkpoint_callback,
  callbacks=[early_stopping_callback],
  max_epochs=N_EPOCHS,
  gpus=1,
  progress_bar_refresh_rate=30
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [64]:
trainer.fit(lmmodel, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name | Type            | Params
-----------------------------------------
0 | bert | BertForMaskedLM | 108 M 
-----------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
433.366   Total estimated model params size (MB)
/ssd003/home/elau/Conv_BERT/BERT_MLM_env/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Validation sanity check: 0it [00:00, ?it/s]

/ssd003/home/elau/Conv_BERT/BERT_MLM_env/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Epoch 0, global step 11: val_loss reached 0.41922 (best 0.41922), saving model to "/ssd003/home/elau/Conv_BERT/BERT_MLM_OOD/checkpoints_OOD/best-checkpoints.ckpt" as top 1


Validating: 0it [00:00, ?it/s]

Epoch 1, global step 23: val_loss reached 0.21792 (best 0.21792), saving model to "/ssd003/home/elau/Conv_BERT/BERT_MLM_OOD/checkpoints_OOD/best-checkpoints.ckpt" as top 1


1

In [71]:
torch.save(lmmodel.state_dict(), 'saved.bin')

## Evaluation

In [78]:
from transformers import AutoModelWithLMHead, AutoTokenizer
import torch
import torch.nn as nn
import pytorch_lightning as pl 
from torch.utils.data import Dataset, DataLoader

import warnings
warnings.filterwarnings("ignore")

def tokenize_sentences(sentences):
    # Tokenize all of the sentences and map the tokens to thier word IDs.
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in sentences:
        encoded_dict = tokenizer.encode_plus(
                            sent,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = 320,           # Pad & truncate all sentences.
                            padding='max_length',
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                    )
        
        # Add the encoded sentence to the list.    
        input_ids.append(encoded_dict['input_ids'])
        
        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks.append(encoded_dict['attention_mask'])

    # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    return input_ids,attention_masks        

Loading BERT tokenizer...


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The BERT model has 202 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (28997, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28997, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
           

In [20]:
from transformers import AutoModelWithLMHead, AutoTokenizer, BertTokenizer, BertForMaskedLM
import torch
import torch.nn as nn
import pytorch_lightning as pl 
from torch.utils.data import Dataset, DataLoader

import warnings
warnings.filterwarnings("ignore")

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
special_tokens_dict = {'additional_special_tokens': ['[BLK]']}
tokenizer.add_special_tokens(special_tokens_dict)

class BertPred(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = BertForMaskedLM.from_pretrained('bert-base-cased')
        self.bert.resize_token_embeddings(len(tokenizer))
        print_model_info(self.bert)

    def forward(self, input_ids, attention_mask, labels=None):
        output = self.bert(input_ids, attention_mask=attention_mask)
        loss = 0
#         if labels is not None:
#             loss = self.criterion(output, labels)
        return loss, output

new_model = BertPred()
new_model.load_state_dict(torch.load('saved.bin'))
new_model.eval()

chk_path = "checkpoint_path" + "/best-checkpoints.ckpt"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

new_model.to(device)
model = new_model

correct = 0


for i in range(len(test_questions)):
    string = test_questions[i]
    intent = test_intents[i]

    string = string + " the reason of the question is " + " [MASK]"*6
    
    inputs = tokenizer.encode_plus(
      string,
      add_special_tokens=True,
      max_length=300,
      return_token_type_ids=False,
      padding="max_length",
      return_attention_mask=True,
      return_tensors='pt',
    )
    
    mask_token_index = torch.where(inputs['input_ids'] == tokenizer.mask_token_id)[1]
    token_logits = model(inputs['input_ids'].to(device), inputs['attention_mask'].to(device))[1]
    print(test_questions[i])
    pred_intent = ''
    for ele in mask_token_index: 
        mask_token_logits = token_logits[0][0, [ele], :]
        top_5_tokens = torch.topk(mask_token_logits, 1, dim=1).indices[0].tolist()
        for token in top_5_tokens:
            if tokenizer.decode([token]) != "[BLK]":
                pred_intent = pred_intent + " " + (tokenizer.decode([token]))
    print("Predicted Intent:" + pred_intent)
    print("Actual Intent: " + intent)
    if pred_intent == intent:
        correct += 1
    print('='*80)

Loading BERT tokenizer...


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The BERT model has 202 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (28997, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

Predicted Intent: lost withdrawal withdrawal
Actual Intent: lost or stolen phone
What all am I required to show for the identity check?
Predicted Intent: verify identity identity
Actual Intent: verify my identity
Nothing goes through on my card.
Predicted Intent: card card card
Actual Intent: card not working
I will need to get a new card soon, how do I order?
Predicted Intent: edit re withdrawal
Actual Intent: expire card
How do I change my information?
Predicted Intent: edit [PAD]
Actual Intent: edit personal details
The rate of exchange isn't right for my cash withdrawal.
Predicted Intent: cash withdrawal withdrawal withdrawal
Actual Intent: wrong cash withdrawal
when will i receive a US transfer
Predicted Intent: transfer transfer withdrawal
Actual Intent: transfer timing
My card still hasn't arrived after 2 weeks. Is it lost?
Predicted Intent: lost withdrawal withdrawal
Actual Intent: card arrival
can I have a non virtual card?
Predicted Intent: virtual virtual virtual
Actual Inte

Predicted Intent: visa visa ##card ##card
Actual Intent: visa or mastercard
What steps do I need to take to change my card PIN?
Predicted Intent: edit withdrawal [PAD]
Actual Intent: change pin
I want to change my address.
Predicted Intent: edit [PAD]
Actual Intent: edit personal details
What countries will my card be supported in?
Predicted Intent: support support support
Actual Intent: country support
Which cash machines will allow me to change my PIN?
Predicted Intent: edit cash withdrawal
Actual Intent: change pin
What can I use a virtual disposable card for?
Predicted Intent: virtual withdrawal withdrawal
Actual Intent: card limits
When will the transfer go through?
Predicted Intent: transfer transfer withdrawal
Actual Intent: pending transfer
How long is it going to take for my funds to show in my account?
Predicted Intent: card funds funds
Actual Intent: transfer timing
Is there anywhere my card will not be accepted?
Predicted Intent: declined acceptance
Actual Intent: card acce

...

Predicted Intent: verify balance withdrawal
Actual Intent: cash balance not updated
Where can I pay with my debit or credit card?
Predicted Intent: visa withdrawal card
Actual Intent: card acceptance
When will my transfer be available in my account.
Predicted Intent: transfer transfer withdrawal
Actual Intent: transfer balance not updated
When will my card arrive?
Predicted Intent: card card card
Actual Intent: card delivery estimate
Are there ways to transfer money into my account?
Predicted Intent: transfer transfer withdrawal
Actual Intent: transfer into account
Do you offer multiple cards for the same account?
Predicted Intent: card card card
Actual Intent: getting spare card
I got less cash than the one I specified in the ATM
Predicted Intent: cash cash withdrawal
Actual Intent: wrong cash received
I didn't receive all the cash I asked for
Predicted Intent: receiving withdrawal withdrawal
Actual Intent: wrong cash received
Can you tell me the total cost of a transfer?
Predicted In

Predicted Intent: visa virtual
Actual Intent: change pin
I think someone else is using my card.  There are transactions I didn't make.
Predicted Intent: verify verify [PAD]
Actual Intent: compromised card
I need to validate the source of my funds.
Predicted Intent: verify funds funds
Actual Intent: verify source of funds
My statement has a dollar I have been charged showing up on it.
Predicted Intent: transfer charged charged
Actual Intent: extra charge on statement
Can you help me find top up by cash deposit?
Predicted Intent: verify cash withdrawal
Actual Intent: top up cash
It appears my card expires next month, can I order a new one?
Predicted Intent: ex withdrawal withdrawal
Actual Intent: expire card
I did a payment but the app reverted it
Predicted Intent: edit withdrawal payment
Actual Intent: reverted card payment?
Help!  My wallet was stolen and someone is taking money out.  I need this money!  What can I do?
Predicted Intent: declined stolen stolen
Actual Intent: cash withdr

Predicted Intent: transfer transfer transfer
Actual Intent: failed transfer
Can my daughter open an account?
Predicted Intent: edit [PAD] [PAD]
Actual Intent: age limit
I was wondering how I could have two charges for the same item happen more than once in a 7 day period. Is there anyway I could get this corrected asap.
Predicted Intent: wrong charge charge
Actual Intent: transaction charged twice
My contactless payments have not been completed successfully.
Predicted Intent: verify payment payment
Actual Intent: contactless not working
how to change name
Predicted Intent: edit [PAD] [PAD]
Actual Intent: edit personal details
Where is the virtual card?
Predicted Intent: virtual virtual virtual
Actual Intent: getting virtual card
If I don't want a physical card can i get a virtual version?
Predicted Intent: card physical physical
Actual Intent: getting virtual card
From where are you getting your exchange rates?
Predicted Intent: exchange rate rate
Actual Intent: exchange rate
They char

Predicted Intent: edit fee fee charged
Actual Intent: extra charge on statement
If I need a PIN for my card, where is it located?
Predicted Intent: edit app [PAD]
Actual Intent: get physical card
Can you help me reset my password?
Predicted Intent: re reverted [PAD]
Actual Intent: passcode forgotten
Please terminate my account.
Predicted Intent: edit [PAD] withdrawal
Actual Intent: terminate account
I received my new card, how is it activated?
Predicted Intent: card withdrawal withdrawal
Actual Intent: activate my card
How do i get my salary in the account?
Predicted Intent: edit withdrawal withdrawal
Actual Intent: receiving money
If my kids wanted to use your service how old would they have to be?
Predicted Intent: edit [PAD]
Actual Intent: age limit
I have not received the amount of cash i was supposed to.
Predicted Intent: receiving cash cash
Actual Intent: wrong cash received
I don't know how to top up my Google pay.
Predicted Intent: topping topping [PAD]
Actual Intent: app pay
I

Predicted Intent: card virtual card
Actual Intent: virtual card not working
Can I exchange currencies?
Predicted Intent: exchange currency currency currency
Actual Intent: fiat currency support
what's the process for getting a disposable virtual card?
Predicted Intent: card virtual [PAD]
Actual Intent: get virtual card
Can I top up using a cheque?
Predicted Intent: top up
Actual Intent: top up cash
I'm unsure of a withdrawl in my statement.
Predicted Intent: verify withdrawal withdrawal
Actual Intent: cash withdrawal not recognised
About this card PIN?
Predicted Intent: card card card
Actual Intent: get physical card
There are transactions I didn't make, someone else must have used my card.
Predicted Intent: verify [PAD] [PAD]
Actual Intent: compromised card
Can you explain the transfer fee to me?
Predicted Intent: transfer transfer fee
Actual Intent: transfer fee charged
0
